In [10]:
# !pip3 install torch torchvision torchaudio
# !pip install transformers -U
# !pip install sentencepiece
# !pip install py-rouge
# !pip install rouge-metric
# !pip install evaluate
!pip install bert_score


     ---------------------------------------- 61.1/61.1 kB ? eta 0:00:00



[notice] A new release of pip available: 22.2.2 -> 23.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Import Modules

In [3]:
import numpy as np
import pandas as pd

Import Data and Create new Dataframe

In [6]:
data=pd.read_csv('/latest_news_scraper/2.text_preprocessing/combined/combined_file.csv')

new_df = data[['url','clean_text']]
new_df

,url,clean_text
0,https://edition.cnn.com/2023/02/28/sport/los-a...,during the lakers' 27-point comeback victory o...
1,https://edition.cnn.com/2023/02/16/sport/lesle...,it's while running in the moors and hills of h...
2,https://edition.cnn.com/2023/02/27/sport/damia...,damian lillard set an nba record in his monste...
3,https://edition.cnn.com/2023/02/27/football/su...,betrayal has formed part of european politics ...
4,https://edition.cnn.com/2023/02/22/football/pa...,the palestinian team has never taken part in a...
...,...,...
216,https://www.channelnewsasia.com/sport/fa-cup-q...,london : fa cup quarter-final draw made on wed...
217,https://www.channelnewsasia.com/sport/man-unit...,"manchester, england :manchester united fought ..."
218,https://www.channelnewsasia.com/sport/formula-...,statistics for sunday's season-opening bahrain...
219,https://www.channelnewsasia.com/sport/prop-hao...,prop mohamed haouas will miss france's remaini...


Import Pegasus Model using cnn_dailymail dataset

In [64]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

model_1 = AutoModelForSeq2SeqLM.from_pretrained('google/pegasus-cnn_dailymail')
tokenizer_1 = AutoTokenizer.from_pretrained('google/pegasus-cnn_dailymail')


Create Function for Summarization with Pegasus model

In [68]:
def pegasus_summarize(text):
    tokens_input_1 = tokenizer_1.encode("summarize: "+ text, return_tensors='pt', max_length=512, truncation=True)
    ids_1 = model_1.generate(tokens_input_1, min_length=80, max_length=120)
    summary_1 = tokenizer_1.decode(ids_1[0], skip_special_tokens=True)

    return summary_1



In [70]:
text_summary1=new_df['clean_text']
text_summary1

text_summary1=[pegasus_summarize(doc) for doc in text_summary1]

text_summary1


["lebron james injured his foot during Sunday's win over the dallas mavericks.<n>The 38-year-old was seen walking gingerly after the game despite being able to finish the win.<n>Reports say the 19-time all-star could miss several weeks, but is awaiting further tests to determine the full extent of the injury.<n>The lakers remain in danger of missing the playoffs for the second successive season.",
 'lesley paterson\'s "all quiet on the western front" is an adaptation of erich maria remarque\'s anti-war novel of the same name published in 1929.<n>The film focuses on the tragedy and ruin of war, eschewing any notions of heroism or adventure.<n>felix kammerer stars as paul bumer in "all quiet on the western front"',
 'damian lillard racked up 71 points in 39 minutes, including 13 three-pointers, as the trail blazers beat the rockets 121-114.<n>lillard became the first player in history to score more than 70 points in under 40 minutes and just the eighth player to hit the 70-point mark.<n>

In [71]:
df_2 = new_df.copy()
df_2['pegasus_summary']= text_summary1
df_2.to_csv('pegasus_summary2.csv', index=False)


Pegasus Model + Change Size of Token Returned

In [99]:
from transformers import PegasusForConditionalGeneration, PegasusTokenizer
import torch

def pegasus_summarize_3(text):
    tokenizer_model = PegasusTokenizer.from_pretrained("google/pegasus-xsum")
    loaded_model = PegasusForConditionalGeneration.from_pretrained("google/pegasus-xsum")
    tokens = tokenizer_model(text, truncation=True, padding="longest", return_tensors="pt")
    summary = loaded_model.generate(**tokens, min_length=20, max_length=120)
    text=tokenizer_model.decode(summary[0], skip_special_tokens=True)
    return text


text_summary3=new_df['clean_text']

text_summary3=[pegasus_summarize_3(doc) for doc in text_summary3]

text_summary3

["One of the NBA's all-time greats has been ruled out for the rest of the season with a foot injury.",
 '"If we can hold up a mirror to what\'s going on to try and prevent more action from happening, that truly is my goal as a storyteller to effect change."',
 "It's not every day you see a player score more than 70 points in a game.",
 'betrayal, betrayal, betrayal, betrayal, betrayal, betrayal, betrayal, betrayal, betrayal, betrayal, betrayal, betrayal, betrayal, betrayal, betrayal',
 'palestinians are celebrating after their national football team qualified for the 2023 Fifa World Cup.',
 'Novak djokovic has broken the record for the most weeks at the top of the world rankings held by Steffi graf.',
 "As the world's top boxers prepare to compete at the world championships in Doha, the sport's future is in doubt.",
 "He was one of Wales' best-known rugby players and was a member of the all black team.",
 'The world number one novak djokovic is hopeful for "a positive result" on us ten

Pysummarization

In [ ]:
!pip install pysummarization

In [111]:
from pysummarization.nlpbase.auto_abstractor import AutoAbstractor
from pysummarization.tokenizabledoc.simple_tokenizer import SimpleTokenizer
from pysummarization.abstractabledoc.top_n_rank_abstractor import TopNRankAbstractor

def pysummarize(text):
    auto_abstractor = AutoAbstractor()
    # Set tokenizer.
    auto_abstractor.tokenizable_doc = SimpleTokenizer()
    # Set delimiter for making a list of sentence.
    auto_abstractor.delimiter_list = [".", "\n"]
    # Object of abstracting and filtering document.
    abstractable_doc = TopNRankAbstractor()
    # Summarize document.
    result_dict = auto_abstractor.summarize(text, abstractable_doc)
    return("".join(result_dict['summarize_result']))

# Output result.

# for sentence in result_dict["summarize_result"]:
#     print(sentence)

text_summary4=new_df['clean_text']

text_summary4=[pysummarize(doc) for doc in text_summary4]

text_summary4

['during the lakers\' 27-point comeback victory over the dallas mavericks on sunday, tv cameras caught lebron james saying he heard his foot "pop" when landing from a layup.\n the 38-year-old was seen walking gingerly after the game despite being able to finish the win.\njust as the lakers seem to be building some momentum with a three-game winning streak, with their stars shining and the new additions playing their part, james looks set to be in for a spell on the sidelines.\nmultiple reports say the 19-time all-star could miss several weeks, but is awaiting further tests to determine the full extent of the injury.\njames posted a photo of his injured foot to his instagram story on monday, after it was announced he would miss the match against the memphis grizzlies, along with the caption: "fkn sucks!!!!"just another day in \'laker land\' -- and it couldn\'t have come at a worse time.\nso the season began as many predicted, with the lakers winning just two of their first 12 games.\nmi

In [112]:
df_3 = new_df.copy()
df_3['pysummarize']= text_summary4
df_3.to_csv('pysummarize.csv', index=False)

Pegasus Model using Auto Tokenizer

In [117]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

model = AutoModelForSeq2SeqLM.from_pretrained('google/pegasus-xsum')
tokenizer = AutoTokenizer.from_pretrained('google/pegasus-xsum')

tokens_input = tokenizer.encode("summarize: "+ new_df['clean_text'][0], return_tensors='pt', truncation=True)
ids = model.generate(tokens_input, min_length=50, max_length=120)
summary = tokenizer.decode(ids[0], skip_special_tokens=True)

print(summary)

One of the NBA's all-time greats has been ruled out for the rest of the season after suffering a foot injury in the Los Angeles lakers' win over the Memphis Grizzlies at the Staples Center in Los Angeles, on Sunday.


In [118]:
def sum_2(text):
    tokens_input = tokenizer.encode("summarize: "+ text, return_tensors='pt', truncation=True)
    ids = model.generate(tokens_input, min_length=50, max_length=120)
    summary = tokenizer.decode(ids[0], skip_special_tokens=True)
    
    return summary

text_summary2=new_df['clean_text']

text_summary2=[sum_2(doc) for doc in text_summary2]

text_summary2

["One of the NBA's all-time greats has been ruled out for the rest of the season after suffering a foot injury in the Los Angeles lakers' win over the Memphis Grizzlies at the Staples Center in Los Angeles, on Sunday.",
 'As the world marks the 100th anniversary of the end of World War One, the writer and director of one of this year\'s Palme d\'Or contenders talks about the inspiration behind the film\'s opening scene, which she describes as "one of those moments where you know it\'s good and you think: \'oh my god, the uniform\'s more important than the man.\'"',
 "It's not every day you see a player score more than 70 points in a game, but that's exactly what happened in the NBA's all-time classic between the Los Angeles Lakers and Portland Trail blazers at the Staples Center.",
 'The friendship between aleksander eferin and andrea agnelli, then juusvent europe president and club association chairman, was like a "betrayal" between friends, according to the director of a new televisi

In [30]:
new_df.to_csv('pegasus_summaryv3.csv', index=False)

In [119]:
new_df['summary_pegasus']=text_summary2
new_df

C:\Users\syafi\AppData\Local\Temp/ipykernel_27104/1414468045.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['summary_pegasus']=text_summary2


,url,clean_text,summary_pegasus
0,https://edition.cnn.com/2023/02/28/sport/los-a...,during the lakers' 27-point comeback victory o...,One of the NBA's all-time greats has been rule...
1,https://edition.cnn.com/2023/02/16/sport/lesle...,it's while running in the moors and hills of h...,As the world marks the 100th anniversary of th...
2,https://edition.cnn.com/2023/02/27/sport/damia...,damian lillard set an nba record in his monste...,It's not every day you see a player score more...
3,https://edition.cnn.com/2023/02/27/football/su...,betrayal has formed part of european politics ...,The friendship between aleksander eferin and a...
4,https://edition.cnn.com/2023/02/22/football/pa...,the palestinian team has never taken part in a...,palestinians are celebrating after their team ...
...,...,...,...
216,https://www.channelnewsasia.com/sport/fa-cup-q...,london : fa cup quarter-final draw made on wed...,Previews and team news for the FA Cup quarter-...
217,https://www.channelnewsasia.com/sport/man-unit...,"manchester, england :manchester united fought ...","manchester united, sheffield united andgrimsby..."
218,https://www.channelnewsasia.com/sport/formula-...,statistics for sunday's season-opening bahrain...,Formula 1 heads to Bahrain for the season-open...
219,https://www.channelnewsasia.com/sport/prop-hao...,prop mohamed haouas will miss france's remaini...,France prop mohamed haouas will miss the rest ...


# Model Evaluation

### Evaluation of Pegasus using Rouge Metrics

In [15]:
combined_data=pd.read_csv('../abstractive/combined_pegasus_goldtruth.csv')

combined_data.dropna(inplace=True)

In [16]:
len(combined_data)

96

In [6]:
from rouge_metric import PyRouge

rouge = PyRouge(rouge_n=(1, 2,3, 4), rouge_l=True, rouge_w=True,
                rouge_w_weight=1.2, rouge_s=True, rouge_su=True, skip_gap=4)
scores = rouge.evaluate(combined_data['pegasus summary'], combined_data['gold truth'])

In [116]:
scores

{'rouge-1': {'r': 0.06543429193782492,
  'p': 0.0008063830048924492,
  'f': 0.0015931329504556661},
 'rouge-2': {'r': 0.0, 'p': 0.0, 'f': 0.0},
 'rouge-3': {'r': 0.0, 'p': 0.0, 'f': 0.0},
 'rouge-4': {'r': 0.0, 'p': 0.0, 'f': 0.0},
 'rouge-l': {'r': 0.06543429193782492,
  'p': 0.0008063830048924492,
  'f': 0.0015931329504556661},
 'rouge-w-1.2': {'r': 0.09837900446202387,
  'p': 0.001249405062120077,
  'f': 0.0024674734198461535},
 'rouge-s4': {'r': 0.0, 'p': 0.0, 'f': 0.0},
 'rouge-su4': {'r': 0.0, 'p': 0.0, 'f': 0.0}}

### Evaluation of Pegasus using BERT Score

In [28]:
from evaluate import load
bertscore = load("bertscore")
results = bertscore.compute(predictions=combined_data['pegasus summary'], references=combined_data['gold truth'], lang="en")
print(results)

{'precision': [0.8446191549301147, 0.8394047617912292, 0.8190465569496155, 0.8665133714675903, 0.8495010137557983, 0.8388718366622925, 0.8636717796325684, 0.845450222492218, 0.8815006017684937, 0.8604516386985779, 0.8297386169433594, 0.8220810890197754, 0.861603319644928, 0.8829523324966431, 0.8431980013847351, 0.8455687165260315, 0.8390985727310181, 0.8382314443588257, 0.8369397521018982, 0.8661029934883118, 0.9009738564491272, 0.8686971068382263, 0.8452743291854858, 0.849420428276062, 0.7448346614837646, 0.8035792112350464, 0.8902730345726013, 0.8607081770896912, 0.8647435307502747, 0.8460294008255005, 0.8679599165916443, 0.8629059791564941, 0.706386387348175, 0.8647406101226807, 0.8642070293426514, 0.8508288264274597, 0.8823255896568298, 0.851731538772583, 0.8586738109588623, 0.8008962869644165, 0.8486719131469727, 0.8834292888641357, 0.8361267447471619, 0.8560142517089844, 0.8622965812683105, 0.8772598505020142, 0.8383579254150391, 0.8342308402061462, 0.7851136326789856, 0.68159389

As the results are given for each row of the dataframe, the codes below is the average of all rows.

#### Precision

In [36]:
total_precision=sum(results['precision'])
p_length=len(results['precision'])
precision=total_precision/p_length
print(round(precision,4))

0.8406


#### Recall

In [38]:
total_recall=sum(results['recall'])
r_length=len(results['recall'])
recall =total_recall/r_length
print(round(recall,4))

0.8457


#### F1 Score

In [39]:
total_f1=sum(results['f1'])
f1_length=len(results['f1'])
f1 =total_f1/f1_length
print(round(f1,4))

0.8429
